In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
df = pd.read_csv('/content/data_1D.csv')

In [ ]:
df.head()

,Unnamed: 0,index,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,0,9320,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0
1,1,3964,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1
2,2,8551,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0
3,3,12366,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0
4,4,6271,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0


# Text Preprocessing

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'index'])
df

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,USER wkwkwkw akhirnya antek Amerika ini pasang...,0,0,0,0,0,0,0,0,0
1,Terlalu suuzon nih rezim URL,1,0,1,0,0,0,0,0,1
2,"Mau tanya sama guntur romli, Bener gak neh :;",0,0,0,0,0,0,0,0,0
3,Genosida Muslim Rohingya oleh Teroris Budha My...,1,0,0,1,1,0,0,0,0
4,USER USER SubhanAllah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3288,USER USER USER USER USER USER USER REZIM SALES...,1,1,0,1,0,0,0,0,1
3289,RT USER: Bukan main dah ah! Wkwkwk. URL,0,0,0,0,0,0,0,0,0
3290,RT USER: USER USER Ruhut tambah gila Tun \xf0\...,1,1,1,0,0,0,0,0,1
3291,Gubernur jaman now harus cerdas ; #PilgubJaten...,0,0,0,0,0,0,0,0,0


In [ ]:
#casefolding
df['Tweet'] = df['Tweet'].str.lower()
def casefold_if_str(text):
    if isinstance(text, str):
        return text.casefold()
    else:
        return text

df['Tweet'] = df['Tweet'].apply(casefold_if_str)

#removing number
df['Tweet'] = df['Tweet'].replace(r'\b\d+\b', '', regex=True)

#removing punctuation
punctuation = '[' + string.punctuation + ']'
df['Tweet'] = df['Tweet'].replace(punctuation, ' ', regex=True)

#removing symbol
import re
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["                     # Begin character set
        "\U0001F600-\U0001F64F" # Emoticons
        "\U0001F300-\U0001F5FF" # Miscellaneous Symbols and Pictographs
        "\U0001F680-\U0001F6FF" # Transport and Map Symbols
        "\U0001F1E0-\U0001F1FF" # Flags (iOS)
        "\U00002702-\U000027B0" # Dingbats
        "\U000024C2-\U0001F251" # Enclosed characters
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Apply the function to the 'text' column
df['Tweet'] = df['Tweet'].apply(remove_emojis)

#remove double white space
df['Tweet'] = df['Tweet'].str.replace(r'\s{2,}', ' ', regex=True)

df


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,user wkwkwkw akhirnya antek amerika ini pasang...,0,0,0,0,0,0,0,0,0
1,terlalu suuzon nih rezim url,1,0,1,0,0,0,0,0,1
2,mau tanya sama guntur romli bener gak neh,0,0,0,0,0,0,0,0,0
3,genosida muslim rohingya oleh teroris budha my...,1,0,0,1,1,0,0,0,0
4,user user subhanallah sekolah di sekolah krist...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3288,user user user user user user user rezim sales...,1,1,0,1,0,0,0,0,1
3289,rt user bukan main dah ah wkwkwk url,0,0,0,0,0,0,0,0,0
3290,rt user user user ruhut tambah gila tun \xf0\x...,1,1,1,0,0,0,0,0,1
3291,gubernur jaman now harus cerdas pilgubjateng20...,0,0,0,0,0,0,0,0,0


In [ ]:
import nltk
from nltk.corpus import stopwords

# Download Indonesian stopwords if not already downloaded
nltk.download('stopwords')
stopwords_indo = set(stopwords.words('indonesian'))

# Remove stopwords using Indonesian stopwords
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords_indo]))

df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,user wkwkwkw antek amerika pasang badan krn sa...,0,0,0,0,0,0,0,0,0
1,suuzon nih rezim url,1,0,1,0,0,0,0,0,1
2,guntur romli bener gak neh,0,0,0,0,0,0,0,0,0
3,genosida muslim rohingya teroris budha myanmar...,1,0,0,1,1,0,0,0,0
4,user user subhanallah sekolah sekolah kristen ...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3288,user user user user user user user rezim sales...,1,1,0,1,0,0,0,0,1
3289,rt user main dah ah wkwkwk url,0,0,0,0,0,0,0,0,0
3290,rt user user user ruhut gila tun \xf0\x9f\x98\...,1,1,1,0,0,0,0,0,1
3291,gubernur jaman now cerdas pilgubjateng2018 pil...,0,0,0,0,0,0,0,0,0


# Text Representation

In [ ]:
x = df['Tweet']
y = df[['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']]

In [ ]:
# Split the data into train, validation, and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
x_train

577             kalo anies menang tahap lengserkan jokowi
3127    mantan anggota komisi viii jazuli juwaini pind...
946     user cebong ikut2 an gelas nya gambar priode c...
1217    user memaksa rakyat berfikir teroris pesan pol...
2488    pengacara setya novanto â ngototâ pemanggilan ...
                              ...                        
2190    hm menyampah tengok live fb paan khair din tu ...
1170    user aahh winwen syial lesbi my typical of my ...
1418    user user tenggelamkan rezim yg berkuasa sa ad...
2465    user user user user user cebong dinasehati bu ...
3114    ebonk suka bgt hebohin hal2 receh pohon plasti...
Name: Tweet, Length: 2107, dtype: object

In [ ]:
y_train

,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
577,1,0,1,0,0,0,0,0,1
3127,0,0,0,0,0,0,0,0,0
946,0,1,0,0,0,0,0,0,0
1217,1,1,0,1,0,0,0,0,1
2488,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2190,1,0,1,0,1,1,0,0,0
1170,0,1,0,0,0,0,0,0,0
1418,0,0,0,0,0,0,0,0,0
2465,1,1,0,1,0,0,0,0,1


In [ ]:
# Recheck the data size
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))
print(len(x_valid))
print(len(y_valid))

2107
2107
659
659
527
527


In [ ]:
!pip install transformers
from transformers import RobertaTokenizer, TFAutoModel, AdamWeightDecay

In [ ]:
# Tokenize using Roberta
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Tokenize function using RoBERTa tokenizer
def tokenize(tokenizer, texts, max_length):
    input_ids = []
    attention_masks = []

    for text in texts:
        tokens = tokenizer.encode_plus(text, max_length=max_length,
                                       truncation=True, padding='max_length',
                                       add_special_tokens=True, return_attention_mask=True,
                                       return_token_type_ids=False, return_tensors='tf')

        input_ids.append(np.asarray(tokens["input_ids"]).reshape(max_length,))
        attention_masks.append(np.asarray(tokens["attention_mask"]).reshape(max_length,))

    return np.asarray(input_ids), np.asarray(attention_masks)

In [ ]:
# State the length
SEQ_LEN = 100

In [ ]:
# Tokenize train, validation, and test data
train_input_ids, train_attention_masks = tokenize(tokenizer, x_train, SEQ_LEN)
valid_input_ids, valid_attention_masks = tokenize(tokenizer, x_valid, SEQ_LEN)
test_input_ids, test_attention_masks = tokenize(tokenizer, x_test, SEQ_LEN)

# Modeling

In [ ]:
roberta = TFAutoModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [ ]:
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = roberta(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.LSTM(128)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(512)(X)
X = tf.keras.layers.Activation("relu")(X)
X = tf.keras.layers.Dense(512)(X)
X = tf.keras.layers.Dropout(0.3)(X)
y = tf.keras.layers.Dense(9, activation='sigmoid', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# Freeze the LSTM layer (layer 1)
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 100)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 100)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_roberta_model (TFRobert  TFBaseModelOutputWithPooli   1246456   ['input_ids[0][0]',           
 aModel)                     ngAndCrossAttentions(last_   32         'attention_mask[0][0]']      
                             hidden_state=(None, 100, 7                                       

Explanation: <br>
- input_ids represents the input sequence IDs
- mask represents the attention mask for the input sequence
- SEQ_LEN is the length of the input sequences.
- The Roberta model (roberta) will generate embeddings for the input sequences (input_ids) with the provided attention mask (mask). [0] is used to extract the first element from the output tuple returned by roberta, which contains the embeddings.
- An LSTM layer with 128 units is applied to the embeddings to capture sequential dependencies in the data.
- Batch normalization is applied to normalize the activations of the previous LSTM layer.
- Two dense layers with 512 units each are added, followed by ReLU activation and a dropout layer with a dropout rate of 0.3 to prevent overfitting.
- A dense layer with 9 units and sigmoid activation is added to produce the output probabilities for each of the 9 classes in a multi-label classification scenario.
- Create a Keras Model using the inputs (input_ids and mask) and the output (y). <br>

LSTM layer is freezed to show that the weights will be fixed during training. This means that the LSTM layer will not be updated during backpropagation, and its weights will remain unchanged. Thereby it could utilize the LSTM as a fixed feature extractor. This setup assumes that the features learned by the LSTM from the embeddings are sufficient for the task. Hence, we only need to fine-tune the subsequent layers (BatchNormalization, Dense, Activation, Dropout, and the final Dense layer).

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the optimizer
optimizer = 'adam'  # Use the string identifier 'adam'

# Define the loss and accuracy metric
loss = tf.keras.losses.BinaryCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy('accuracy')

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit((train_input_ids, train_attention_masks),y_train, validation_data=((valid_input_ids,valid_attention_masks), y_valid), epochs=15, batch_size=32)

Epoch 1/15
66/66 [==============================] - 59s 235ms/step - loss: 0.4306 - accuracy: 0.7877 - val_loss: 0.4542 - val_accuracy: 0.8079
Epoch 2/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3902 - accuracy: 0.8112 - val_loss: 0.4331 - val_accuracy: 0.8151
Epoch 3/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3738 - accuracy: 0.8181 - val_loss: 0.4418 - val_accuracy: 0.8196
Epoch 4/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3686 - accuracy: 0.8214 - val_loss: 0.3790 - val_accuracy: 0.8225
Epoch 5/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3575 - accuracy: 0.8242 - val_loss: 0.3683 - val_accuracy: 0.8253
Epoch 6/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3561 - accuracy: 0.8261 - val_loss: 0.3573 - val_accuracy: 0.8270
Epoch 7/15
66/66 [==============================] - 6s 84ms/step - loss: 0.3481 - accuracy: 0.8286 - val_loss: 0.3768 - val_accuracy: 0.8294
Epoch 8/15


In [ ]:
from sklearn.metrics import classification_report

# Function to predict and threshold probabilities
def predict_threshold(model, input_ids, attention_masks, threshold=0.5):
    # Predict probabilities
    y_pred_proba = model.predict((input_ids, attention_masks))

    # Apply thresholding
    y_pred_binary = (y_pred_proba > threshold).astype(int)

    return y_pred_binary

# Threshold for prediction
threshold = 0.5

In [ ]:
# Predictions for train sets
train_pred = predict_threshold(model, train_input_ids, train_attention_masks, threshold)

# Classification report for train set
print("Train Set Classification Report:")
print(classification_report(y_train, train_pred))

66/66 [==============================] - 14s 62ms/step
Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.84      0.70       902
           1       0.65      0.87      0.75       828
           2       0.51      0.58      0.55       585
           3       0.75      0.13      0.22       317
           4       0.83      0.14      0.24       140
           5       0.90      0.33      0.48        86
           6       1.00      0.05      0.10        39
           7       0.00      0.00      0.00        35
           8       0.50      0.73      0.59       621

   micro avg       0.58      0.66      0.62      3553
   macro avg       0.64      0.41      0.40      3553
weighted avg       0.61      0.66      0.59      3553
 samples avg       0.46      0.41      0.41      3553



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for validation sets
valid_pred = predict_threshold(model, valid_input_ids, valid_attention_masks, threshold)

# Classification report for validation set
print("\nValidation Set Classification Report:")
print(classification_report(y_valid, valid_pred))

17/17 [==============================] - 1s 60ms/step

Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.74      0.60       203
           1       0.58      0.79      0.67       208
           2       0.45      0.55      0.50       141
           3       0.57      0.06      0.12        62
           4       0.33      0.07      0.12        28
           5       0.75      0.13      0.22        23
           6       0.00      0.00      0.00        15
           7       0.00      0.00      0.00        14
           8       0.43      0.66      0.52       136

   micro avg       0.50      0.59      0.54       830
   macro avg       0.40      0.34      0.30       830
weighted avg       0.49      0.59      0.50       830
 samples avg       0.41      0.36      0.35       830



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for test sets
test_pred = predict_threshold(model, test_input_ids, test_attention_masks, threshold)

# Classification report for test set
print("\nTest Set Classification Report:")
print(classification_report(y_test, test_pred))

21/21 [==============================] - 1s 60ms/step

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.72      0.60       284
           1       0.61      0.75      0.67       264
           2       0.40      0.50      0.44       187
           3       0.73      0.08      0.15        97
           4       0.57      0.09      0.16        44
           5       0.80      0.13      0.22        31
           6       0.00      0.00      0.00        21
           7       0.00      0.00      0.00        20
           8       0.38      0.65      0.48       180

   micro avg       0.49      0.56      0.52      1128
   macro avg       0.44      0.32      0.30      1128
weighted avg       0.50      0.56      0.48      1128
 samples avg       0.40      0.35      0.35      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Based on the values ​​above, it can be seen that the accuracy is still not good, especially in validation and testing.

## Setting the best threshold

### Threshold = 0.5

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# Get multilabel confusion matrix
cm = multilabel_confusion_matrix(y_train, train_pred)
print("Confusion Matrix Shape:", cm.shape)

# Print each confusion matrix
for i in range(len(cm)):
    print(f"Confusion Matrix for Label {i}:")
    print(cm[i])
    print()

Confusion Matrix Shape: (9, 2, 2)
Confusion Matrix for Label 0:
[[697 508]
 [146 756]]

Confusion Matrix for Label 1:
[[899 380]
 [109 719]]

Confusion Matrix for Label 2:
[[1199  323]
 [ 244  341]]

Confusion Matrix for Label 3:
[[1777   13]
 [ 277   40]]

Confusion Matrix for Label 4:
[[1963    4]
 [ 120   20]]

Confusion Matrix for Label 5:
[[2018    3]
 [  58   28]]

Confusion Matrix for Label 6:
[[2068    0]
 [  37    2]]

Confusion Matrix for Label 7:
[[2072    0]
 [  35    0]]

Confusion Matrix for Label 8:
[[1025  461]
 [ 169  452]]



In the confusion matrix above, it can be seen that there are many wrong predictions, especially in labels 0, 1, and 2. Even though the initial model  fitting results (history) went well, where the accuracy was high and the loss was small. Besides, it is possible that the threshold of 0.5 is not entirely appropriate to use in this case. Therefore, a search for the right threshold value will be carried out in the next stage.

### Threshold = 0.2

In [ ]:
threshold = 0.2
# Predictions for train sets
train_pred = predict_threshold(model, train_input_ids, train_attention_masks, threshold)

# Classification report for train set
print("Train Set Classification Report:")
print(classification_report(y_train, train_pred))

66/66 [==============================] - 4s 61ms/step
Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.98      0.67       902
           1       0.50      0.97      0.66       828
           2       0.39      0.88      0.54       585
           3       0.59      0.35      0.44       317
           4       0.64      0.33      0.43       140
           5       0.73      0.38      0.50        86
           6       0.67      0.05      0.10        39
           7       0.50      0.03      0.05        35
           8       0.38      0.94      0.55       621

   micro avg       0.46      0.84      0.59      3553
   macro avg       0.55      0.55      0.44      3553
weighted avg       0.48      0.84      0.58      3553
 samples avg       0.41      0.49      0.43      3553



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for validation sets
valid_pred = predict_threshold(model, valid_input_ids, valid_attention_masks, threshold)

# Classification report for validation set
print("\nValidation Set Classification Report:")
print(classification_report(y_valid, valid_pred))

17/17 [==============================] - 1s 61ms/step

Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.92      0.59       203
           1       0.49      0.95      0.65       208
           2       0.33      0.77      0.47       141
           3       0.30      0.15      0.20        62
           4       0.37      0.25      0.30        28
           5       0.57      0.17      0.27        23
           6       0.00      0.00      0.00        15
           7       0.50      0.07      0.12        14
           8       0.33      0.88      0.48       136

   micro avg       0.40      0.76      0.53       830
   macro avg       0.37      0.46      0.34       830
weighted avg       0.40      0.76      0.50       830
 samples avg       0.38      0.44      0.38       830



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for test sets
test_pred = predict_threshold(model, test_input_ids, test_attention_masks, threshold)

# Classification report for test set
print("\nTest Set Classification Report:")
print(classification_report(y_test, test_pred))

21/21 [==============================] - 1s 61ms/step

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.90      0.62       284
           1       0.49      0.92      0.64       264
           2       0.34      0.76      0.47       187
           3       0.48      0.24      0.32        97
           4       0.28      0.11      0.16        44
           5       0.67      0.19      0.30        31
           6       0.00      0.00      0.00        21
           7       0.00      0.00      0.00        20
           8       0.32      0.86      0.47       180

   micro avg       0.41      0.74      0.53      1128
   macro avg       0.34      0.44      0.33      1128
weighted avg       0.41      0.74      0.50      1128
 samples avg       0.39      0.44      0.39      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


From the results above we can see that the model performance is more or less the same as before. It's just that now there is no overfitting because the model has the same accuracy (around 0.50).

# Modeling (ii)

Try modifying the batch_size

In [ ]:
history = model.fit((train_input_ids, train_attention_masks),y_train, validation_data=((valid_input_ids,valid_attention_masks), y_valid), epochs=15, batch_size=64)

Epoch 1/15
33/33 [==============================] - 5s 147ms/step - loss: 0.2827 - accuracy: 0.8409 - val_loss: 0.4779 - val_accuracy: 0.8419
Epoch 2/15
33/33 [==============================] - 5s 146ms/step - loss: 0.2709 - accuracy: 0.8429 - val_loss: 0.3849 - val_accuracy: 0.8437
Epoch 3/15
33/33 [==============================] - 5s 144ms/step - loss: 0.2611 - accuracy: 0.8447 - val_loss: 0.4238 - val_accuracy: 0.8455
Epoch 4/15
33/33 [==============================] - 5s 143ms/step - loss: 0.2597 - accuracy: 0.8464 - val_loss: 0.4213 - val_accuracy: 0.8471
Epoch 5/15
33/33 [==============================] - 5s 143ms/step - loss: 0.2526 - accuracy: 0.8479 - val_loss: 0.3939 - val_accuracy: 0.8487
Epoch 6/15
33/33 [==============================] - 5s 143ms/step - loss: 0.2484 - accuracy: 0.8494 - val_loss: 0.3958 - val_accuracy: 0.8503
Epoch 7/15
33/33 [==============================] - 5s 143ms/step - loss: 0.2474 - accuracy: 0.8511 - val_loss: 0.4367 - val_accuracy: 0.8518
Epoch 

### threshold = 0.5

In [ ]:
threshold = 0.5

# Predictions for train sets
train_pred = predict_threshold(model, train_input_ids, train_attention_masks, threshold)

# Classification report for train set
print("Train Set Classification Report:")
print(classification_report(y_train, train_pred))

66/66 [==============================] - 4s 61ms/step
Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.86       902
           1       0.90      0.65      0.75       828
           2       0.83      0.63      0.71       585
           3       0.75      0.62      0.68       317
           4       1.00      0.41      0.58       140
           5       0.84      0.85      0.84        86
           6       0.73      0.28      0.41        39
           7       0.92      0.31      0.47        35
           8       0.82      0.73      0.77       621

   micro avg       0.86      0.69      0.77      3553
   macro avg       0.85      0.59      0.68      3553
weighted avg       0.86      0.69      0.76      3553
 samples avg       0.45      0.39      0.40      3553



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for validation sets
valid_pred = predict_threshold(model, valid_input_ids, valid_attention_masks, threshold)

# Classification report for validation set
print("\nValidation Set Classification Report:")
print(classification_report(y_valid, valid_pred))

17/17 [==============================] - 1s 60ms/step

Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.59      0.61       203
           1       0.80      0.49      0.60       208
           2       0.55      0.38      0.45       141
           3       0.35      0.29      0.32        62
           4       0.50      0.18      0.26        28
           5       0.44      0.30      0.36        23
           6       0.00      0.00      0.00        15
           7       0.00      0.00      0.00        14
           8       0.56      0.46      0.51       136

   micro avg       0.61      0.44      0.51       830
   macro avg       0.43      0.30      0.35       830
weighted avg       0.60      0.44      0.50       830
 samples avg       0.29      0.25      0.25       830



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for test sets
test_pred = predict_threshold(model, test_input_ids, test_attention_masks, threshold)

# Classification report for test set
print("\nTest Set Classification Report:")
print(classification_report(y_test, test_pred))

21/21 [==============================] - 1s 60ms/step

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       284
           1       0.74      0.49      0.59       264
           2       0.60      0.43      0.50       187
           3       0.48      0.33      0.39        97
           4       0.62      0.18      0.28        44
           5       0.64      0.29      0.40        31
           6       0.00      0.00      0.00        21
           7       0.00      0.00      0.00        20
           8       0.53      0.46      0.49       180

   micro avg       0.63      0.45      0.53      1128
   macro avg       0.48      0.31      0.37      1128
weighted avg       0.61      0.45      0.51      1128
 samples avg       0.31      0.26      0.26      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In the classification report, it can be seen that the training has good performance with an accuracy of 77%. This value indicates that the training has run well. However, for validation and testing, the accuracy value is only around 50%. A difference of more than 20% indicates overfitting in this model.

### Threshold = 0.2

In [ ]:
# Threshold for binary predictions
threshold = 0.2

# Predictions for train sets
train_pred = predict_threshold(model, train_input_ids, train_attention_masks, threshold)

# Classification report for train set
print("Train Set Classification Report:")
print(classification_report(y_train, train_pred))

66/66 [==============================] - 4s 61ms/step
Train Set Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.94      0.86       902
           1       0.78      0.83      0.81       828
           2       0.67      0.81      0.73       585
           3       0.56      0.83      0.67       317
           4       0.95      0.56      0.70       140
           5       0.69      0.93      0.79        86
           6       0.65      0.51      0.57        39
           7       0.70      0.46      0.55        35
           8       0.68      0.89      0.77       621

   micro avg       0.72      0.85      0.78      3553
   macro avg       0.72      0.75      0.72      3553
weighted avg       0.73      0.85      0.78      3553
 samples avg       0.46      0.48      0.46      3553



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for validation sets
valid_pred = predict_threshold(model, valid_input_ids, valid_attention_masks, threshold)

# Classification report for validation set
print("\nValidation Set Classification Report:")
print(classification_report(y_valid, valid_pred))

17/17 [==============================] - 1s 60ms/step

Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.72      0.63       203
           1       0.68      0.72      0.70       208
           2       0.51      0.60      0.55       141
           3       0.22      0.39      0.28        62
           4       0.47      0.25      0.33        28
           5       0.42      0.43      0.43        23
           6       0.00      0.00      0.00        15
           7       0.33      0.14      0.20        14
           8       0.45      0.70      0.55       136

   micro avg       0.51      0.63      0.56       830
   macro avg       0.40      0.44      0.41       830
weighted avg       0.52      0.63      0.56       830
 samples avg       0.34      0.35      0.32       830



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predictions for test sets
test_pred = predict_threshold(model, test_input_ids, test_attention_masks, threshold)

# Classification report for test set
print("\nTest Set Classification Report:")
print(classification_report(y_test, test_pred))

21/21 [==============================] - 1s 61ms/step

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.75      0.68       284
           1       0.68      0.71      0.69       264
           2       0.48      0.60      0.53       187
           3       0.35      0.51      0.42        97
           4       0.62      0.30      0.40        44
           5       0.46      0.35      0.40        31
           6       0.20      0.05      0.08        21
           7       0.00      0.00      0.00        20
           8       0.47      0.68      0.56       180

   micro avg       0.55      0.63      0.58      1128
   macro avg       0.43      0.44      0.42      1128
weighted avg       0.54      0.63      0.57      1128
 samples avg       0.37      0.37      0.34      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In the classification report, it can be seen that the training has a slightly better performance than the previous threshold, with an accuracy of 78%. This value indicates that the training has run well. However, for validation and testing, the accuracy value is only close to 60%. A roughly 20% difference indicates overfitting in this model.

# Conclusion

The results of the history model is good but the results of the classification report are surprisingly not, where the accuracy is not good and there are empty values ​​​​on certain labels. Therefore, I did several ways to deal with this problem, including trying to adjust the threshold. because it is possible that the threshold of 0.5 which is a common starting point, might not be optimal for all labels. If the threshold is too high or too low, it can lead to all predictions being either 0 or 1 for some labels, resulting in empty confusion matrices (e.g., all predicted values ​​​​are 0s, and there are no 1s to match the true labels). Besides, I've ensure that the data is properly balanced across labels. Because if some labels are heavily skewed (e.g., very few positive instances), then applying a fixed threshold may not capture the true distribution of predictions needed for accurate classification.